In [46]:
import numpy as np

In [47]:
G = 'B' # Type of classical algebraic group, Input: string 'A','B','C', or 'D'
l = 3 # Rank, Input: positive integer
n = 2*l+1 # Square matrix dimension for type B

In [48]:
"""
Make matrices for the root vectors for the positive roots in type B (i.e., the root vectors in the nilradical).
Store the root vector matrices in a dictionary called rvecs where the keys are the eigenvector names given on page 131
of Erdmann and the values are the eigenvectors as numpy arrays

"mat_type" is the eigenvector matrix form given on page 131 of Erdmann
"indices" is a list of the i,j indices
"matrix" is the eigenvector corresponding to the root of specified type and indices
"coords" are the coordinates of the positive nonzero matrix entry, to be used for the coordinate function

Parameters: rank l

Returns: dictionary

"""
def make_rvecs_B(l):
    rvecs = {}

    M = np.zeros((2*l+1,2*l+1),dtype = int)
    # Add root vectors b_i for roots of the form epsilon_i to rvecs
    for i in np.arange(1,l+1,1):
        # b_ij
        M[i,0] = 1
        M[0,l+i] = -1
        hilo = "b" + str(i)
        coef = np.zeros(l, dtype = int) # Array of length the number of roots in the base
        coef[i-1:l] = 1
        coef = tuple(coef)
        rvecs.setdefault(coef, {"matrix":M, "coords":[i,0]}) # index 1 \leq i \leq l
        M = np.zeros((2*l+1,2*l+1),dtype = int)
    for i in np.arange(1,l,1):
        for j in np.arange(1,l+1,1):
            # p_ij
            if i<j:
                M[i,l+j] = 1
                M[j,l+i] = -1
                hilo = "p" + str(i) + str(j)
                coef = np.zeros(l, dtype = int) # Array of length the number of roots in the base
                coef[i-1:j] = 1
                coef[j-1:l] = 2
                coef = tuple(coef)
                rvecs.setdefault(coef, {"matrix":M, "coords":[i,l+j]}) # indices 1 \leq i < j \leq l
                M = np.zeros((2*l+1,2*l+1),dtype = int)
            # m_ij
            if i != j and i<j:
                M[i,j] = 1
                M[l+j,l+i] = -1
                hilo = "m" + str(i) + str(j)
                coef = np.zeros(l, dtype = int) # Array of length the number of roots in the base
                coef[i-1:j-1] = 1 
                coef = tuple(coef)
                rvecs.setdefault(coef, {"matrix":M, "coords":[i,j]})
                M = np.zeros((2*l+1,2*l+1), dtype = int)
    return rvecs

rvecs = make_rvecs_B(l)
print(rvecs)

{(1, 1, 1): {'matrix': array([[ 0,  0,  0,  0, -1,  0,  0],
       [ 1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0]]), 'coords': [1, 0]}, (0, 1, 1): {'matrix': array([[ 0,  0,  0,  0,  0, -1,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0]]), 'coords': [2, 0]}, (0, 0, 1): {'matrix': array([[ 0,  0,  0,  0,  0,  0, -1],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0]]), 'coords': [3, 0]}, (1, 2, 2): {'matrix': array([[ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  1,  0

In [49]:
"""
Fix a sequence of the positive roots with a (partial) ordering induced by increasing root height.

Use the Sage function positive_roots_by_height(increasing=True)

Parameters: Group type G, string
            Rank, integer

Returns: List of roots as tuples
"""
def make_root_order(G,l):
    L = RootSystem([G,l]).root_lattice()
    roots = [tuple(root.to_vector()) for root in L.positive_roots_by_height()]
    return roots

roots = make_root_order(G,l)
num_roots = len(roots) # Store the number of positive roots for future loops

In [50]:
"""
VERSION WHERE THE ENTRIES ARE THE DIFFERENT THAN THOSE IN THE NILRADICAL (r INSTEAD OF n)
Compute the root spaces $\mathfrak{g}_\beta=\mathbb{C}e_\beta$

Notes:
This could be combined with the function that makes the nilradical
The "DeprecationWarning: invalid escape sequence \m" that returns when this cell is run in Jupyter is because of the 
backslash character used in the TeX for this comment

Parameters: rvecs

Returns: rspaces: dictionary with keys the roots as tuples and values the root spaces
"""
def make_root_spaces(rvecs):
    rspaces = {} # Create a dictionary rspaces with keys the roots as stored in rvecs and values the root spaces k*e_alpha
    num = len(rvecs) # Set the number of iterations to equal the number of positive roots; we'll need this for most things in this function
    syms = np.array([var("r"+str(list(rvecs.keys())[i]).replace("(", "").replace(")", "").replace(" ", "").replace(",", "")) for i in range(num)], dtype = object) # Name some not-necessarily-distinct field elements
    # Note that rvecs[list(rvecs.keys())[1]]['matrix'] extracts the eigenvector matrix from rvecs
    mats = [np.dot(syms[i],rvecs[list(rvecs.keys())[i]]['matrix']) for i in range(num)] # Multiply the root vectors by an arbitrary field element to get the root spaces
    for i in range(num):
        rspaces.setdefault(list(rvecs.keys())[i],mats[i])
    return rspaces

rspaces = make_root_spaces(rvecs)

<>:13: DeprecationWarning: invalid escape sequence \m
<>:13: DeprecationWarning: invalid escape sequence \m
<>:13: DeprecationWarning: invalid escape sequence \m
<ipython-input-50-969feb4a3bad>:13: DeprecationWarning: invalid escape sequence \m
  """


In [51]:
"""
VERSION WHERE THE ENTRIES ARE THE SAME AS IN THE NILRADICAL
Compute the root spaces $\mathfrak{g}_\beta=\mathbb{C}e_\beta$

Notes:
This could be combined with the function that makes the nilradical
The "DeprecationWarning: invalid escape sequence \m" that returns when this cell is run in Jupyter is because of the 
backslash character used in the TeX for this comment

Parameters: rvecs

Returns: rspaces: dictionary with keys the roots as tuples and values the root spaces
"""
def make_root_spaces(rvecs):
    rspaces = {} # Create a dictionary rspaces with keys the roots as stored in rvecs and values the root spaces k*e_alpha
    num = len(rvecs) # Set the number of iterations to equal the number of positive roots; we'll need this for most things in this function
    syms = np.array([var("n"+str(list(rvecs.keys())[i]).replace("(", "").replace(")", "").replace(" ", "").replace(",", "")) for i in range(num)], dtype = object) # Name some not-necessarily-distinct field elements
    # Note that rvecs[list(rvecs.keys())[1]]['matrix'] extracts the eigenvector matrix from rvecs
    mats = [np.dot(syms[i],rvecs[list(rvecs.keys())[i]]['matrix']) for i in range(num)] # Multiply the root vectors by an arbitrary field element to get the root spaces
    for i in range(num):
        rspaces.setdefault(list(rvecs.keys())[i],mats[i])
    return rspaces

rspaces = make_root_spaces(rvecs)

<>:13: DeprecationWarning: invalid escape sequence \m
<>:13: DeprecationWarning: invalid escape sequence \m
<>:13: DeprecationWarning: invalid escape sequence \m
<ipython-input-51-6312676660ef>:13: DeprecationWarning: invalid escape sequence \m
  """


In [52]:
"""
Compute a matrix representing an arbitrary element of the nilradical

Input: rvecs
Returns: numpy.ndarray ; Arbitrary element of the nilradical
"""
def make_nilrad(rvecs):
    rspaces = {} # Create a dictionary rspaces with keys the roots as stored in rvecs and values the root spaces k*e_alpha
    
    num = len(rvecs) # Set the number of iterations to equal the number of positive roots; we'll need this for most things in this function
    syms = np.array([var("n"+str(list(rvecs.keys())[i]).replace("(", "").replace(")", "").replace(" ", "").replace(",", "")) for i in range(num)], dtype = object) # Name some not-necessarily-distinct field elements
    # Note that rvecs[list(rvecs.keys())[1]]['matrix'] extracts the eigenvector matrix from rvecs
    mats = [np.dot(syms[i],rvecs[list(rvecs.keys())[i]]['matrix']) for i in range(num)] # Multiply the root vectors by an arbitrary field element to get the root spaces
    nil_rad = sum(mats) # Set an arbitrary element of the nilradical equal to the sum of the root spaces
    
    for i in range(num):
        rspaces.setdefault(list(rvecs.keys())[i],mats[i])
    
    return nil_rad #,rspaces, mats

nil_rad = make_nilrad(rvecs)
nil_rad

array([[0, 0, 0, 0, -n111, -n011, -n001],
       [n111, 0, n100, n110, 0, n122, n112],
       [n011, 0, 0, n010, -n122, 0, n012],
       [n001, 0, 0, 0, -n112, -n012, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, -n100, 0, 0],
       [0, 0, 0, 0, -n110, -n010, 0]], dtype=object)

In [53]:
np.zeros((n,n), dtype = int)

array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]])

In [54]:
"""
Compute the NT-submodules $\mathfrak{m}_i$
"""
def make_NT_submodules(n,roots,rspaces):
    mi_dict = {'m0': np.zeros((n,n), dtype = int)}
    for i in range(len(roots)):
        hilo = "m" + str(i+1)
        mi = rspaces[roots[i]]
        for k in range(i+1, len(roots)):
            mi += rspaces[roots[k]]
            #print(k,roots[k])

        mi_dict.setdefault(hilo,mi)
    return mi_dict

make_NT_submodules(7,roots,rspaces)

<>:3: DeprecationWarning: invalid escape sequence \m
<>:3: DeprecationWarning: invalid escape sequence \m
<>:3: DeprecationWarning: invalid escape sequence \m
<ipython-input-54-1fb64ab2ccc4>:3: DeprecationWarning: invalid escape sequence \m
  """


{'m0': array([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]]),
 'm1': array([[0, 0, 0, 0, -n111, -n011, -n001],
        [n111, 0, n100, n110, 0, n122, n112],
        [n011, 0, 0, n010, -n122, 0, n012],
        [n001, 0, 0, 0, -n112, -n012, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, -n100, 0, 0],
        [0, 0, 0, 0, -n110, -n010, 0]], dtype=object),
 'm2': array([[0, 0, 0, 0, -n111, -n011, 0],
        [n111, 0, n100, n110, 0, n122, n112],
        [n011, 0, 0, n010, -n122, 0, n012],
        [0, 0, 0, 0, -n112, -n012, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, -n100, 0, 0],
        [0, 0, 0, 0, -n110, -n010, 0]], dtype=object),
 'm3': array([[0, 0, 0, 0, -n111, -n011, 0],
        [n111, 0, n100, n110, 0, n122, n112],
        [n011, 0, 0, 0, -n122, 0, n012],
        [0, 0, 0, 0, -n112, -n012, 0],
       

In [68]:
"""
SCRATCH

Compute the NT-submodules $\mathfrak{m}_i$ as coordinates rather than matrices
"""
mi_dict = {}
for i in range(num_roots):
    hilo = "m" + str(i)
    mi = []
    mi.append(rvecs[roots[i]]['coords'])
    for k in range(i+1, num_roots):
        mi.append(rvecs[roots[k]]['coords'])
        #print(k,roots[k])
    
    mi_dict.setdefault(hilo,mi)
mi_dict

<>:3: DeprecationWarning: invalid escape sequence \m
<>:3: DeprecationWarning: invalid escape sequence \m
<>:3: DeprecationWarning: invalid escape sequence \m
<ipython-input-68-104dbd72c16d>:3: DeprecationWarning: invalid escape sequence \m
  """


{'m0': [[3, 0],
  [2, 3],
  [1, 2],
  [2, 0],
  [1, 3],
  [2, 6],
  [1, 0],
  [1, 6],
  [1, 5]],
 'm1': [[2, 3], [1, 2], [2, 0], [1, 3], [2, 6], [1, 0], [1, 6], [1, 5]],
 'm2': [[1, 2], [2, 0], [1, 3], [2, 6], [1, 0], [1, 6], [1, 5]],
 'm3': [[2, 0], [1, 3], [2, 6], [1, 0], [1, 6], [1, 5]],
 'm4': [[1, 3], [2, 6], [1, 0], [1, 6], [1, 5]],
 'm5': [[2, 6], [1, 0], [1, 6], [1, 5]],
 'm6': [[1, 0], [1, 6], [1, 5]],
 'm7': [[1, 6], [1, 5]],
 'm8': [[1, 5]]}

In [103]:
# SCRATCH
m_list = []
for level in range(num_roots):
    mat = nil_rad
    #print(mat)
    for rspace in range(len(mi_dict['m'+str(level)])):
        print(mi_dict['m'+str(level)])
        coords = mi_dict['m'+str(level)][rspace]
        print(coords)
        mat[coords[0],coords[1]] = 0
        m_list.append(mat)

[[3, 0], [2, 3], [1, 2], [2, 0], [1, 3], [2, 6], [1, 0], [1, 6], [1, 5]]
[3, 0]
[[3, 0], [2, 3], [1, 2], [2, 0], [1, 3], [2, 6], [1, 0], [1, 6], [1, 5]]
[2, 3]
[[3, 0], [2, 3], [1, 2], [2, 0], [1, 3], [2, 6], [1, 0], [1, 6], [1, 5]]
[1, 2]
[[3, 0], [2, 3], [1, 2], [2, 0], [1, 3], [2, 6], [1, 0], [1, 6], [1, 5]]
[2, 0]
[[3, 0], [2, 3], [1, 2], [2, 0], [1, 3], [2, 6], [1, 0], [1, 6], [1, 5]]
[1, 3]
[[3, 0], [2, 3], [1, 2], [2, 0], [1, 3], [2, 6], [1, 0], [1, 6], [1, 5]]
[2, 6]
[[3, 0], [2, 3], [1, 2], [2, 0], [1, 3], [2, 6], [1, 0], [1, 6], [1, 5]]
[1, 0]
[[3, 0], [2, 3], [1, 2], [2, 0], [1, 3], [2, 6], [1, 0], [1, 6], [1, 5]]
[1, 6]
[[3, 0], [2, 3], [1, 2], [2, 0], [1, 3], [2, 6], [1, 0], [1, 6], [1, 5]]
[1, 5]
[[2, 3], [1, 2], [2, 0], [1, 3], [2, 6], [1, 0], [1, 6], [1, 5]]
[2, 3]
[[2, 3], [1, 2], [2, 0], [1, 3], [2, 6], [1, 0], [1, 6], [1, 5]]
[1, 2]
[[2, 3], [1, 2], [2, 0], [1, 3], [2, 6], [1, 0], [1, 6], [1, 5]]
[2, 0]
[[2, 3], [1, 2], [2, 0], [1, 3], [2, 6], [1, 0], [1, 6], [1, 5]]

# Scratch

In [145]:
list(rvecs.keys())[1]

(0, 1, 1)

In [162]:
mats[1]

array([[0, 0, 0, 0, 0, -n011, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [n011, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]], dtype=object)

In [169]:
(mats[0]+mats[1])@nil_rad-nil_rad@(mats[0]+mats[1])

array([[0, 0, 0, 0, n011*n100, 0, 0],
       [-n011*n100, 0, 0, 0, 0, 0, -n001*n111],
       [0, 0, 0, 0, 0, 0, -n001*n011],
       [0, 0, 0, 0, n001*n111, n001*n011, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]], dtype=object)

In [144]:
nil_rad@rvecs[list(rvecs.keys())[1]]['matrix']-rvecs[list(rvecs.keys())[1]]['matrix']@nil_rad

array([[0, 0, 0, 0, -n4, 0, 0],
       [n4, 0, 0, 0, 0, -n0, 0],
       [0, 0, 0, 0, n0, 0, n2],
       [0, 0, 0, 0, 0, -n2, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]], dtype=object)

In [52]:
g = LieAlgebra(QQ, cartan_type=[G,l]) #, representation="matrix"

In [91]:
B3 = RootSystem(["B",3]).root_space()
roots=list(B3.positive_roots())
roots

[alpha[1],
 alpha[2],
 alpha[3],
 alpha[1] + alpha[2],
 alpha[2] + 2*alpha[3],
 alpha[2] + alpha[3],
 alpha[1] + alpha[2] + 2*alpha[3],
 alpha[1] + alpha[2] + alpha[3],
 alpha[1] + 2*alpha[2] + 2*alpha[3]]

In [53]:
g

Lie algebra of ['B', 3] in the Chevalley basis

In [19]:
g.lie_algebra_generators()

Finite family {alpha[1]: E[alpha[1]], alpha[2]: E[alpha[2]], alpha[3]: E[alpha[3]], -alpha[1]: E[-alpha[1]], -alpha[2]: E[-alpha[2]], -alpha[3]: E[-alpha[3]], alphacheck[1]: h1, alphacheck[2]: h2, alphacheck[3]: h3}

In [20]:
g.inject_variables()

Defining e1, e2, e3, f1, f2, f3, h1, h2, h3


In [94]:
print(e1)

E[alpha[1]]


In [67]:
n = LieAlgebra(QQ, cartan_type=['A',4], nilpotent=True)

In [ ]:
# Fix an enumeration of the positive roots s.t. ht(\beta_i)\leq ht(\beta_{i+1})



In [100]:
so4 = lie_algebras.so(QQ, 5, representation='matrix'); so4
so4.gens()

(
[ 0  1  0  0  0]  [ 0  0  0  0  0]  [ 0  0  0  0  0]  [ 0  0  0  0  0]
[ 0  0  0  0  0]  [ 0  0  0  0  2]  [ 1  0  0  0  0]  [ 0  0  0  0  0]
[ 0  0  0  0  0]  [ 0  0  0  0  0]  [ 0  0  0 -1  0]  [ 0  0  0  0  0]
[ 0  0 -1  0  0]  [ 0  0  0  0  0]  [ 0  0  0  0  0]  [ 0  0  0  0 -1]
[ 0  0  0  0  0], [ 0  0  0 -2  0], [ 0  0  0  0  0], [ 0  1  0  0  0],

[ 1  0  0  0  0]  [ 0  0  0  0  0]
[ 0 -1  0  0  0]  [ 0  2  0  0  0]
[ 0  0 -1  0  0]  [ 0  0  0  0  0]
[ 0  0  0  1  0]  [ 0  0  0 -2  0]
[ 0  0  0  0  0], [ 0  0  0  0  0]
)